In [1]:
%pip install -r /home/aalab/Desktop/tripoFT/TripoSR/requirements.txt

  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-jg99a0e5
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-jg99a0e5
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3aef8afa5f21b113afc4f4ea148baee850cbd472
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.2.2+cu121
12.1


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import sys
import os

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR
from omegaconf import OmegaConf


/home/aalab/Desktop/tripoFT/TripoSR/venv_name/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import sys
import os
import torch
from omegaconf import OmegaConf

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR

# Load the configuration file
config_path = 'TripoSR/config.yaml'
cfg = OmegaConf.load(config_path)

# Manually set `tokenizer.num_channels` before initializing other components
cfg.backbone.in_channels = cfg.tokenizer.num_channels

# Initialize the TSR model with the loaded and updated configuration
model = TSR(cfg=cfg)

/home/aalab/Desktop/tripoFT/TripoSR/venv_name/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/aalab/Desktop/tripoFT/TripoSR/venv_name/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
model.to(device)

TSR(
  (image_tokenizer): DINOSingleImageTokenizer(
    (model): ViTModel(
      (embeddings): ViTEmbeddings(
        (patch_embeddings): ViTPatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): ViTEncoder(
        (layer): ModuleList(
          (0-11): 12 x ViTLayer(
            (attention): ViTAttention(
              (attention): ViTSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (

In [7]:
import sys
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class MeshDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.npz')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)
        vertices = data['vertices']
        faces = data['faces']

        # Convert to torch tensors
        vertices = torch.tensor(vertices, dtype=torch.float32)
        faces = torch.tensor(faces, dtype=torch.long)  # Indices should be long integers

        return vertices, faces

def custom_collate_fn(batch):
    vertices_list = [item[0] for item in batch]
    faces_list = [item[1] for item in batch]

    # Padding vertices
    max_vertices = max([v.shape[0] for v in vertices_list])
    padded_vertices = [torch.cat([v, torch.zeros(max_vertices - v.shape[0], 3)], dim=0) for v in vertices_list]
    
    # Padding faces
    max_faces = max([f.shape[0] for f in faces_list])
    padded_faces = [torch.cat([f, torch.zeros(max_faces - f.shape[0], 3, dtype=torch.long)], dim=0) for f in faces_list]

    vertices_tensor = torch.stack(padded_vertices)
    faces_tensor = torch.stack(padded_faces)

    return vertices_tensor, faces_tensor

# Paths to your data directories
train_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# Create dataset instances
train_dataset = MeshDataset(train_dir)
val_dataset = MeshDataset(val_dir)
test_dataset = MeshDataset(test_dir)

# Function to move data to GPU
def move_to_gpu(batch):
    vertices, faces = batch
    return vertices.cuda(), faces.cuda()

# Create data loaders with custom collate function
batch_size = 3  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

# Example of iterating through the train_loader
for vertices, faces in train_loader:
    print(vertices.shape, faces.shape)
    # Add your training code here
    break


torch.Size([3, 151286, 3]) torch.Size([3, 242136, 3])


In [8]:
import torch
import torch.nn as nn

class ChamferDistance(nn.Module):
    def forward(self, points1, points2):
        batch_size, num_points, _ = points1.size()
        dist1 = torch.cdist(points1, points2)
        dist2 = torch.cdist(points2, points1)
        
        loss1 = dist1.min(dim=2)[0].mean(dim=1)
        loss2 = dist2.min(dim=2)[0].mean(dim=1)
        
        loss = (loss1 + loss2).mean()
        return loss

# Example usage
loss_fn = ChamferDistance()


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a loss function using Chamfer Distance
class ChamferDistance(nn.Module):
    def forward(self, points1, points2):
        batch_size, num_points, _ = points1.size()
        dist1 = torch.cdist(points1, points2)
        dist2 = torch.cdist(points2, points1)

        loss1 = dist1.min(dim=2)[0].mean(dim=1)
        loss2 = dist2.min(dim=2)[0].mean(dim=1)

        loss = (loss1 + loss2).mean()
        return loss

loss_fn = ChamferDistance()

# Define an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate as needed

# Function to transform model output to point cloud
def transform_output_to_point_cloud(output):
    batch_size, _, D, H, W = output.shape
    # Example transformation: Flatten the volumetric grid and sample points
    output_flat = output.view(batch_size, 3, -1)  # Flatten the spatial dimensions
    output_points = output_flat.permute(0, 2, 1)  # Reorder to shape [batch_size, num_points, 3]
    return output_points


In [10]:
# Wrapper function to handle model call with device
def call_model_with_device(model, vertices, device):
    return model(vertices, device)

In [29]:
from torch.cuda.amp import autocast, GradScaler

# Training function with gradient accumulation and mixed precision
def train(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device, accumulation_steps=4):
    scaler = GradScaler()  # For mixed precision

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (vertices, faces) in enumerate(train_loader):
            vertices, faces = vertices.to(device), faces.to(device)

            optimizer.zero_grad()
            batch_size = vertices.size(0)
            if batch_size > 1:
                vertices = vertices[:1]
                faces = faces[:1]
                
            with autocast():  # Mixed precision context
                model_output = call_model_with_device(model, vertices, device)  # Call model with device argument
                outputs = transform_output_to_point_cloud(model_output)  # Transform to point cloud

                # Check shapes of outputs and vertices
                if outputs.shape[1] != vertices.shape[1]:
                    # Handle shape mismatch if necessary (e.g., downsample or upsample outputs)
                    outputs = outputs[:, :vertices.shape[1], :]

                loss = loss_fn(outputs, vertices)
                loss = loss / accumulation_steps  # Normalize the loss

            scaler.scale(loss).backward()  # Scaled loss backward
            
            if (i + 1) % accumulation_steps == 0:  # Perform step every accumulation_steps
                scaler.step(optimizer)  # Optimizer step
                scaler.update()  # Update scaler
                optimizer.zero_grad()

            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')

        # # Validation step (optional)
        # model.eval()
        # with torch.no_grad():
        #     val_loss = 0
        #     for vertices, faces in val_loader:
        #         vertices, faces = vertices.to(device), faces.to(device)
        #         model_output = call_model_with_device(model, vertices, device)
        #         outputs = transform_output_to_point_cloud(model_output)

        #         if outputs.shape[1] != vertices.shape[1]:
        #             outputs = outputs[:, :vertices.shape[1], :]

        #         val_loss += loss_fn(outputs, vertices).item()
        #     val_loss /= len(val_loader)

        # print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

# Example usage
num_epochs = 5 # Adjust as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.93 GiB of which 28.88 MiB is free. Including non-PyTorch memory, this process has 2.67 GiB memory in use. Of the allocated memory 2.58 GiB is allocated by PyTorch, and 3.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Testing function
def test(model, test_loader, loss_fn, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for vertices, faces in test_loader:
            vertices, faces = vertices.to(device), faces.to(device)
            model_output = call_model_with_device(model, vertices, device)
            outputs = transform_output_to_point_cloud(model_output)

            if outputs.shape[1] != vertices.shape[1]:
                outputs = outputs[:, :vertices.shape[1], :]

            test_loss += loss_fn(outputs, vertices).item()
        test_loss /= len(test_loader)

    print(f'Test Loss: {test_loss:.4f}')

# Example usage
test(model, test_loader, loss_fn, device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.93 GiB of which 24.44 MiB is free. Including non-PyTorch memory, this process has 2.63 GiB memory in use. Of the allocated memory 2.54 GiB is allocated by PyTorch, and 5.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)